In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import gaussian_kde
from scipy.stats import norm
from scipy.stats import multivariate_normal

import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Layer, Dropout, BatchNormalization, LeakyReLU, Lambda
from keras.losses import Loss, mse, MeanSquaredError
from keras.optimizers import Optimizer, Adam
from keras.metrics import Mean
from keras.utils import to_categorical, plot_model, load_img, img_to_array
from keras.callbacks import EarlyStopping
from keras.models import load_model

import tensorflow_probability as tfp

np.random.seed(1234)
tf.random.set_seed(1234)


# Bayesian Neural Network (BNN) - Theory

## Kullback-Leibler (KL) Divergence (Relative Entropy)

The Kullback-Leibler (KL) divergence is a measure that quantifies the difference or "distance" between two probability distributions. It is a powerful tool for comparing and contrasting distributions, which is particularly useful in areas such as information theory, machine learning, and statistical inference. Given two probability distributions, $p(x)$ and $q(x)$, the KL divergence can be defined using the following formula:

$$D_{KL}(p||q) =\mathbb{E}\bigg[\log{\bigg(\frac{p(x)}{q(x)}\bigg)}\bigg] $$

In the discrete case, the KL divergence is computed as:

$$D_{KL}(p||q) = \sum_x p(x)\log{\bigg(\frac{p(x)}{q(x)}\bigg)}$$

For continuous probability distributions, the KL divergence is given by the integral:

$$D_{KL}(p||q) = \int_x p(x)\log{\bigg(\frac{p(x)}{q(x)}\bigg)}$$

It is important to note that the KL divergence is not symmetric, meaning that $D_{KL}(p||q) \neq D_{KL}(q||p)$. The KL divergence can be interpreted as the expected logarithmic difference between the two probability distributions, with the expectation taken over the values of $x$ according to the distribution $p(x)$. In essence, the KL divergence quantifies the amount of information lost when we approximate the true distribution $p(x)$ using the distribution $q(x)$.

**Example 1 - Discrete**

Consider the set $w \in {0:Bad, 1:Good}$ representing the weather, and suppose that the weather being good or bad follows a Bernoulli distribution $w \sim \textbf{Bernoulli}(\theta)$. In a hypothetical scenario, two people are arguing about the chance of the weather being good. The first person says that $\theta_A = 0.8$, and the second person says that $\theta_B = 0.7$. How far apart are their opinions (distributions)?

For a Bernoulli distribution, it can be thought of as a model for the set of possible outcomes of any single experiment that asks a yes–no question. The Bernoulli distribution can be written as

$$\textbf{Bernoulli}(w,\theta) = \theta^w(1-\theta)^{1-w}$$

and the KL divergence as

$$D_{KL}(p||q) = \sum^{1}_{w=0} p(w)\log{\bigg(\frac{p(w)}{q(w)}\bigg)}$$

where the probabilities are $p(w) = \theta_A^w(1-\theta_A)^{1-w}$ and $q(w) = \theta_B^w(1-\theta_B)^{1-w}$ . substituting in the Kl divergence:

$$D_{KL}(p||q) = \sum^{1}_{w=0} \theta_A^w(1-\theta_A)^{1-w}\log{\bigg(\frac{\theta_A^w(1-\theta_A)^{1-w}}{\theta_B^w(1-\theta_B)^{1-w}}\bigg)}$$

Now, let's evaluate the sum for $w=0$ and $w=1$:

$$D_{KL}(p||q) = (1-\theta_A)\log{\bigg(\frac{1-\theta_A}{1-\theta_B}\bigg)} + \theta_A\log{\bigg(\frac{\theta_A}{\theta_B}\bigg)}$$

Now, we can plug in the values $\theta_A = 0.8$ and $\theta_B = 0.7$ to compute the KL divergence:

$$D_{KL}(p||q) \approx 0.02573$$

This value represents the distance between the two opinions (distributions) held by the two people.

In [ ]:
weather_A = tfp.distributions.Bernoulli(probs = 0.8)
weather_B = tfp.distributions.Bernoulli(probs = 0.7)

print('D(p||q)=', float(tfp.distributions.kl_divergence(weather_A, weather_B)))

D(p||q)= 0.025732100009918213


**Example 2 - Continous**

Suppose we're studying the height of adult men in two countries: Country A and Country B.
The height of adult men in Country A follows a Gaussian distribution with mean 175 cm and standard deviation 10 cm. The height of adult men in Country B follows a Gaussian distribution with mean 170 cm and standard deviation 15 cm.

We can use the Kullback-Leibler (KL) divergence to measure the difference between these two distributions. 

First, recall that the KL divergence between two Gaussian distributions $p(x)\sim \mathcal{N}(\mu_p, \sigma_p )$ and $q(x)\sim \mathcal{N}(\mu_q,\sigma_q )$ is 

$$D_{KL}(p||q) = \int_x p(x)\log{\bigg(\frac{p(x)}{q(x)}\bigg)}$$

We can substitute their probability density functions into this integral and simplify the result. A probability density function $f(x) \sim \mathcal{N}(\mu,\sigma )$ for a Gaussian distribution with mean $\mu$ and standard deviation $\sigma$ is given by

$$f(x) = \frac{1}{\sqrt{2\pi \sigma}} \exp\bigg(-\frac{(x-\mu)^2}{2 \sigma^2}\bigg)$$

Substituting the expressions for $p(x) \sim \mathcal{N}(\mu_p,\sigma_p )$ and $q(x)\sim \mathcal{N}(\mu_q,\sigma_q )$  into the KL divergence integral and simplifying yields the formula:

$$D_{KL}(p||q) = \log\left(\frac{\sigma_q}{\sigma_p}\right) + \frac{\sigma_p^2 + (\mu_p - \mu_q)^2}{2\sigma_q^2} - \frac{1}{2}$$

Here, $p(x) \sim \mathcal{N}(175.3,7.1 )$ and $q(x)\sim \mathcal{N}(170.8,6.3)$. Substituting these values into the formula for the KL divergence gives:

$$D_{KL}(P||Q) = \log\left(\frac{6.3}{7.1}\right) + \frac{(7.1)^2 +(175.3−170.8)^2 
}{2(6.3))^2} - \frac{1}{2} \approx 0.183$$

$D_{KL}(P||Q) = 0.183$ indicate the amount of information lost when the distribution for Country B is used to approximate the distribution for Country A. This gives us a notion of how distance this two distribution are from each other.


In [12]:
mu_p, sigma_p = 175, 10
mu_q, sigma_q = 170, 15

# distributions
dist_p = tfp.distributions.Normal(loc=mu_p, scale=sigma_p)
dist_q = tfp.distributions.Normal(loc=mu_q, scale=sigma_q)

# KL divergence
kl_divergence = tfp.distributions.kl_divergence(dist_p, dist_q)
print('D(p||q)=', float(kl_divergence))

D(p||q)= 0.18324309587478638


## Variational Inference (VI) and Evidence Lower Bound (ELBO)

Variational inference is a technique used to make predictions and reason about uncertainty in complex probabilistic models by approximating the posterior distribution of latent variables given observed data. Consider a dataset $D$ and latent variables $z$, which are unobserved variables that capture hidden patterns or underlying structure in the data. The goal is to compute the posterior distribution $p(z|D)$, which represents our updated belief about the latent variables $z$ after observing the dataset $D$. To achieve this, we rely on Bayes' rule:

​$$p(z|D) = \frac{p(D|z) \cdot P(z)}{p(D)} = \frac{p(D|z) \cdot p(z)}{\int p(D|z')p(z')dz'} $$

In the continuous case, we use an integral instead of a summation, as the latent variables have a continuous domain rather than a finite set of discrete events. The integral represents a continuous sum over all possible values of the latent variable within its domain.

To compute the marginal likelihood, $p(D)$, for continuous latent variables, we need to integrate over the entire domain of the latent variables to account for all possible configurations. The integral in the denominator of Bayes' rule, $\int p(D|z')P(z') dz'$, serves to sum the probabilities over the entire continuous domain of the latent variable $z$.

The challenge in directly using Bayes' rule lies in calculating the marginal probability $p(D) = \int p(D|z')p(z') dz'$. This integral can be computationally expensive or even intractable for large or high-dimensional latent variable spaces. To address this issue, variational inference approximates the true posterior distribution with a simpler distribution, making the computation more tractable.

To tackle this problem, let's consider the continuous KL-divergence to find some distribution $q(z')$ to approximate the distribution for the posterior probabilities $p(z'|D)$. The goal is to find a distribution $q(z')$ that minimizes this divergence, hence approximating the true distribution as closely as possible.

$$D_{KL}(q||p) = \int q(z')\log{\bigg(\frac{q(z')}{p(z'|D)}\bigg)}dz'$$

The usage of $q(z')$ instead of $p(z'|D)$ in the integrand is due to that we do not have direct access to the true posterior distribution. While it might seem like we're "putting the cart before the horse" by using the approximating distribution $q(z')$ in the integrand, this approach is justified by the practical benefits it provides in the context of variational inference, providing a more flexible choice for $q(z')$.

Substituting the Bayes' rule $p(z'|D) = \frac{p(D|z')p(z')}{p(D)}$ in the KL divergence:

$$D_{KL}(q||p) = \int q(z')\log{\bigg(\frac{q(z')p(D)}{p(D \cap z')}\bigg)}dz'$$

We have the information about the joint probability $p(D|z')p(z')$, but not about the marginal $p(D)$. So rearranging the integral in terms of probabilities that we have and don't have:

$$D_{KL}(q||p) = \int q(z')\log{\bigg(\frac{q(z')}{p(D \cap z')}\bigg)}dz'+ \int q(z')\log{\bigg( p(D) \bigg)}dz'$$

From the definition of expectation value, if $p(z)$ is a continuous distribution of the random variable $z$ and $g(z)$ is some function of $z$, then $\mathbb{E}[g(z)] = \int g(z)p(z)dz$. So, the KL-divergence can be written as a sum of expectation values over the variable $z'$ from the distribution $q(z')$:

$$D_{KL}(q||p) = \mathbb{E}_{z' \sim q}\bigg[\log{\frac{q(z')}{p(D \cap z')}}\bigg]+ \mathbb{E}_{z' \sim q}[\log{p(D)}]$$

Since $p(D)$ is not dependent on $z$, the expectation $\mathbb{E}_{z' \sim q}[\log{p(D)}]$ simplifies to $\log{p(D)}$. Then, we reverse the fraction in the first expectation and add a minus sign in the logarithm:

$$D_{KL}(q||p) = -\mathbb{E}_{z' \sim q}\bigg[\log{\frac{p(D \cap z')}{q(z')}}\bigg]+\log{p(D)}$$

Let's name the first expectation as $\mathcal{L}(q(z);D) = \mathbb{E}_{z' \sim q}\bigg[\log{\frac{p(D|z')p(z')}{q(z')}}\bigg]$ and write the KL-divergence as:

$$D_{KL}(q||p) = -\mathcal{L}(q(z);D) + \log{p(D)}$$

We know that the KL-divergence must be a non-negative value, as we are interested in measuring the distance between the posterior distribution and the distribution $q(z)$ to approximate the posterior. The term $\log{p(D)}$ is fixed for a given dataset and also negative (log of values between zero and one). For the term $\mathcal{L}(q(z);D)$, we expect negative values to balance the negative term $\log{p(D)}$ when the two distributions are close. The value of $\mathcal{L}(q(z);D)$ is expected to be smaller than the value of the evidence or marginal probability $\log{p(D)}$, and because of this, $\mathcal{L}(q(z);D)$ is called the Evidence Lower Bound (ELBO).

- $\mathcal{L}(q(z);D) = \log{p(D)}$ if only if $D_{KL}(q||p) = 0$
  - **Evidence Lower Bound** (ELBO): $\mathcal{L}(q(z);D) = \mathbb{E}_{z' \sim q}\bigg[\log{\frac{p(D \cap z')}{q(z')}}\bigg]$ 
  - **Evidence** : $\log{p(D)}$

The ELBO can be evaluated because it involves quantities that we do know: the likelihood $p(D|z')$, the prior $p(z')$ and the approximating distribution $q(z')$.

Now we arrive at the concept of Variational inference, where $D_{KL}(q||p)$ is a non-negative value when the distributions are not close and close to zero when the two distributions are approximated. So we are approximating our evidence $\log{p(D)}$ from below, and this is the case where we need to maximize the ELBO term $\mathcal{L}(q(z);D)$ for a certain family of distributions $Q$. The family $Q$ is a set of candidate distributions, and our goal is to find the best distribution $q^*(z)$ within that family that approximates the true posterior distribution $p(z|D)$:

$$q^*(z)  = \operatorname{argmax}_{q \in Q}(\mathcal{L}(q(z);D))$$

For this maximization, we rearrange the ELBO in terms of the KL-Divergence:

$$\mathcal{L}(q(z);D) = - D_{KL}(q||p) +  \log{p(D)}$$

By maximizing the ELBO, we obtain an approximation $q^*(z)$ to the true posterior distribution $p(z|D)$, making the computation more tractable. It suggests that the ELBO is a trade-off between the reconstruction accuracy against the complexity of the variational posterior. The KL divergence term can be interpreted as a measure of the additional information required to express the posterior relative to the prior. As it approaches zero, the posterior is fully obtainable from the prior.

In conclusion, Variational Inference (VI) is a method for approximating the true posterior distribution $p(z|D)$ in complex probabilistic models by optimizing a simpler distribution $q(z)$. The method leverages the KL divergence to measure the dissimilarity between the true posterior and the approximating distribution $q(z)$. The main goal is to minimize the KL divergence, which is equivalent to maximizing the Evidence Lower Bound (ELBO). As we see, the ELBO is a function of $q(z)$ and is related to the KL divergence and the log of the evidence $\log{p(D)}$. By maximizing the ELBO, we find a distribution $q^*(z)$ that approximates the true posterior distribution $p(z|D)$.

### Mean Field Approach

We previously discussed the need to maximize the ELBO term $\mathcal{L}(q(z);D)$ over a certain family of distributions $Q$. The family $Q$ is a set of candidate distributions, and our goal is to find the best distribution $q^*(z)$ within that family to approximate the true posterior distribution $p(z|D)$:

$$q^*(z)  = \operatorname{argmax}_{q \in Q}(\mathcal{L}(q(z);D))$$

However, selecting an appropriate family of distributions $Q$ is not straightforward. To address this challenge, we can employ the Mean Field approach.

The Mean Field approach simplifies the approximation of the true posterior distribution $p(z|D)$ by assuming that the latent variables $z$ are independent given the parameters of the variational distribution. This assumption results in a factorized form of the distribution $q(z)$. Given a set of latent variables $z = \{z_1, z_2, \ldots, z_n\}$, the Mean Field Variational Family defines the family $Q$ of candidate distributions as those that can be factorized as:

$$q(z) = q(z_1, z_2, \ldots, z_n) = q_1(z_1)q_2(z_2)\cdots q_n(z_n) = \prod_{i=1}^{n} q_i(z_i)$$

Here, $q_i(z_i)$ represents the individual variational distributions for each latent variable $z_i$. This factorization is a direct consequence of the independence assumption. By assuming independence, we can express the joint distribution of all latent variables as a product of their individual marginal distributions.

Using the calculus of variations (hence the name "variational"), we can derive the optimal distribution $q_{j}^{{*}}$ for each of the factors $q_{j}$ that minimizes the KL divergence, as described earlier:

$$q_j^*(z_j) = \frac{e^{\mathbb{E}_{i \neq j} [\ln p(D \cap z)] }}{ \int e^{ \mathbb{E}_{i \neq j} [\ln p(D \cap z)] }dz_j}$$

By taking the logarithm of both sides of the equation, we can further simplify the expression:

$$\log{q_j^*(z_j)} = \log{ e^{\mathbb{E}_{i \neq j} [\ln p(D \cap z)] }} - \log{\int e^{ \mathbb{E}_{i \neq j} [\ln p(D \cap z)] }dz_j}$$

where:
$$\log{q_j^*(z_j)} = \mathbb{E}_{i \neq j} [\ln p(D \cap z)] + \text{Constant}$$

This equation provides a computationally tractable way to find the optimal $q_j^*(z_j)$, which is crucial for approximating the true posterior distribution $p(z|D)$.

To calculate the factorized distribution $q(z) = \prod_{i=1}^{n} q_i(z_i)$ using the optimal variational distributions $q_j^*(z_j)$, we can follow these steps:

1. Initialize the variational distributions $q_i(z_i)$ for all latent variables $z_i$. These initializations can be arbitrary, but a common approach is to use Gaussian distributions with random means and variances.

2. Update each variational distribution $q_j^*(z_j)$ using the derived equation:
  $$\log{q_j^*(z_j)} = \mathbb{E}_{i \neq j} [\ln p(D \cap z)] + \text{Constant}$$

1. Iterate through each latent variable $z_j$ and update its corresponding variational distribution $q_j^*(z_j)$ using the equation above. This process is repeated until convergence, i.e., when the variational distributions do not change significantly between iterations. Convergence can be measured using the difference in ELBO values, or by monitoring the change in the variational distributions' parameters.

2. Once the algorithm has converged, you will have the optimal variational distributions $q_j^*(z_j)$ for all latent variables $z_j$. To obtain the factorized distribution $q(z)$, you can simply multiply the individual optimal variational distributions:

$$q(z) = q(z_1, z_2, \ldots, z_n) = q_1^*(z_1)q_2^*(z_2)\cdots q_n^*(z_n) = \prod_{i=1}^{n} q_i^*(z_i)$$

Now, we have the factorized distribution $q(z)$ that approximates the true posterior distribution $p(z|D)$. Note that the mean-field variational approach may not always provide an exact approximation, but it often yields a reasonable approximation that is computationally tractable.

## BNN from Scratch

By leveraging the concepts of Variational Inference (VI), Evidence Lower Bound (ELBO), and the Mean Field approach, Bayesian Neural Networks (BNNs) can approximate the posterior distribution over the network's weights and biases. This enables BNNs to provide uncertainty estimates for their predictions, which can be crucial in tasks that require a measure of confidence in the model's output. These methods work together to transform a traditional Dense Neural Network (DNN) into a Bayesian framework. 

Recall that in traditional neural networks, the parameters (weights and biases) are point estimates obtained through optimization, such as gradient descent. In a BNN, instead of having point estimates for the weights and biases, we place a prior distribution over these parameters. For example, we could use Gaussian priors with zero mean and some predefined variance. This allows us to treat the network parameters as random variables and compute their posterior distribution given observed data.

To compute the posterior distribution of the parameters, we can use Variational Inference (VI) to approximate it with a simpler distribution. The BNN's objective is to minimize the KL-divergence between the true posterior and the approximating distribution, which is equivalent to maximizing the Evidence Lower Bound (ELBO). By employing the Mean Field approach, we assume that the network's parameters are independent given their variational parameters. This results in a factorized form of the distribution, which simplifies the optimization problem.

The Mean Field Variational Family defines the candidate distributions as those that can be factorized as:

$$q(w) = q(w_1, w_2, \ldots, w_n) = q_1(w_1)q_2(w_2)\cdots q_n(w_n) = \prod_{i=1}^{n} q_i(w_i)$$

Here, $w$ represents the network's parameters (weights and biases), and $q_i(w_i)$ represents the individual variational distributions for each parameter. By iteratively updating the variational distributions for each parameter, we obtain an approximation to the true posterior distribution.

In the context of BNNs, we use the reparameterization trick to simplify the gradient estimation during the optimization process. Instead of directly sampling from the variational distribution, we can represent each weight as $w_i = \mu_i + \sigma_i \epsilon_i$, where $\mu_i$ is the mean, $\sigma_i$ is the standard deviation of the distribution, and $\epsilon_i$ is a random noise term usually sampled from a standard normal distribution, i.e., $\epsilon_i \sim \mathcal{N}(0, 1)$. This enables us to sample weights from the variational distribution while keeping the noise term separate, which is useful during the optimization process.


So, in a Bayesian Neural Network (BNN), each weight is characterized by a distribution (often a Gaussian distribution), which is parameterized by a mean and a standard deviation. These parameters are learned from the data during the training process.


Once the algorithm has converged, we can use the approximated posterior distribution of the parameters to make predictions. The output of a BNN is given by an average of the network's outputs for different parameter samples from the approximated posterior distribution. By sampling multiple sets of parameters from the distribution, we can compute an average prediction and obtain an estimate of the prediction uncertainty or draw a distribution from multiples output samples.

In conclusion, BNNs can approximate the posterior distribution of the network's parameters and provide uncertainty estimates for their predictions.The posterior predictive distribution is the distribution of potential outcomes given the observed data. Each time that an input is passed through the BNN, the weights are sampled from their posterior distributions, and these weights are used to generate an output. Thus, each output can be considered a sample from the posterior predictive distribution for that input.

Here I provide a step-by-step guide to implementing a Bayesian Neural Network (BNN) using Variational Inference (VI), Evidence Lower Bound (ELBO), and the Mean Field approach:

1. Define the neural network architecture: Define the architecture of the neural network (number of layers, units per layer, activation functions) that you want to convert into a Bayesian Neural Network (BNN).

2. Initialize the variational distributions: For each weight and bias in the neural network, initialize a variational distribution (typically a Gaussian distribution with random means and variances) that will approximate the true posterior distribution of the network's parameters.

3. Define the model's likelihood: Define the likelihood function $p(D|w)$, which represents the probability of observing the data $D$ given the weights and biases $w$. In a neural network, this likelihood function is typically represented by the output layer's activation function (e.g., softmax for classification).

4. Compute the ELBO: Define the Evidence Lower Bound $\mathcal{L}(q(w);D)$ according to the formula:

    $\mathcal{L}(q(w);D) = \mathbb{E}_{w' \sim q}\bigg[\log{\frac{p(D \cap w')}{q(w')}}\bigg]$

    Here, $q(w')$ is the variational distribution approximating the true posterior distribution of the network's parameters (weights and biases).


5. Optimize the variational distributions: Use an optimization algorithm (e.g., stochastic gradient descent) to update the parameters of the variational distributions by maximizing the ELBO. This step involves iterating through each parameter in the neural network and updating its corresponding variational distribution using the mean field approach. Calculate the optimal variational distributions $q_j^*(w_j)$ with the equation:
   
    $\log{q_j^*(w_j)} = \mathbb{E}_{i \neq j} [\ln p(D \cap w)] + \text{Constant}$

    such that $q^*(w) = \prod_{i=1}^{n} q_i^*(w_i) = \operatorname{argmax}_{{q_1,\cdots,q_n} \in Q}(\mathcal{L}(q(w);D))$.

    Here, $q^*(w)$ represents the optimal variational distribution for the network's weights, which is a product of the individual optimal variational distributions $q_i^*(w_i)$ for each weight $w_i$. The $\operatorname{argmax}$ notation indicates that we are looking for the set of variational distributions ${q_1,\cdots,q_n}$ within the family $Q$ that maximizes the ELBO. In other words, We want to find the individual variational distributions for each weight that, when combined, maximize the ELBO

6. Monitor convergence: Continue updating the variational distributions until the algorithm converges, i.e., when the variational distributions' parameters or the ELBO values do not change significantly between iterations. 


7. Make predictions with uncertainty estimates: Use the optimized variational distributions to make predictions with the BNN. Sample the network's parameters from the optimized variational distributions and perform a forward pass through the network. Repeat this process multiple times to obtain a distribution of predictions, which will provide an estimate of the uncertainty associated with the BNN's predictions.

but the reparameterized version explicitly demonstrates how the random variables are generated, and it may have some computational advantages in certain contexts, such as allowing gradients to flow more directly

In [ ]:
class DenseVariational(Layer):
    def __init__(self, units, activation=None, kl_weight=1e-3, name = 'DenseVariational_rt'):
        """
        Initialize the custom variational dense layer parameters.
        """
        super(DenseVariational, self).__init__(name = name)
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        self.kl_weight = kl_weight

    def xavier(self, shape):
        """
        Xavier initialization for weights and biases, ensuring proper scaling at the start of training.
        """
        return tf.random.truncated_normal(
            shape=shape,
            mean=0.0,
            stddev=np.sqrt(2.0 / sum(shape)))

    def build(self, input_shape):
        """
        Build the layer by defining the variables for means and standard deviations.
        Variational Inference: The initialization of mean and standard deviation for weights and biases forms 
        the basis for approximating their posterior distributions using Gaussian distributions. 
        Mean Field: By treating weights and biases as independent random variables with their separate mean and 
        standard deviation, this approach assumes a mean-field approximation to the true joint posterior distribution.
        """
        d_in = input_shape[-1]
        # Initializing mean and standard deviation for weights and biases to represent their posterior distributions.
        self.w_mean = tf.Variable(self.xavier((d_in, self.units)), name='w_mean')
        self.w_std = tf.Variable(self.xavier((d_in, self.units)) - 6, name='w_std')
        self.b_mean = tf.Variable(self.xavier((self.units,)), name='b_mean')
        self.b_std = tf.Variable(self.xavier((self.units,)) - 6, name='b_std')

    def call(self, x, training = True):
        """
        Perform the forward pass through the layer, either stochastically, if in training mode, or deterministically.
        Variational Inference: Stochastic forward pass involves sampling from approximate posterior distributions.
        """
        if training: # Stochastic forward pass (sampling process)
            
            # Softplus to ensure std is positive
            w_std = tf.nn.softplus(self.w_std)
            b_std = tf.nn.softplus(self.b_std)
            
            # Random noise for sampling from Gaussian distributions
            # Correlation problem: Unique perturbations for each data point
            w_noise = tf.random.normal(shape=self.w_mean.shape)
            b_noise = tf.random.normal(shape=self.b_mean.shape)
            
            # Samples from the posterior distributions for weights and biases
            w_sample = self.w_mean + w_std*w_noise
            b_sample = self.b_mean + b_std*b_noise
            output = x @ w_sample + b_sample

        else: # Deterministic forward pass
            output =  x @ self.w_mean + self.b_mean
        
        if self.activation is not None:
            output = self.activation(output)
            
        return output

    def kl_divergence(self, mean_1, std_1, mean_2, std_2):
        """
        Calculate the Kullback-Leibler (KL) divergence between two Gaussian distributions.
        KL Divergence: The method implements the mathematical expression for KL divergence.
        """
        term1 = tf.math.log(std_2 / std_1)
        term2 = (std_1**2 + (mean_1 - mean_2)**2) / (2 * std_2**2)
        kl = term1 + term2 - 0.5
        return tf.reduce_sum(kl)

    @property
    def loss(self):
        """
        Compute the loss for the layer, considering the Kullback-Leibler (KL) divergence 
        between the distributions of the weights and biases and a standard normal distribution.
        KL Divergence: KL divergence is used to calculate the loss.
        ELBO: The loss represents part of the Evidence Lower Bound.
        """

        # Prior distribution
        prior_mean = 0.0 
        prior_std = 1.0

        w_std = tf.nn.softplus(self.w_std)
        b_std = tf.nn.softplus(self.b_std)
        kl_w = self.kl_divergence(self.w_mean, w_std, prior_mean, prior_std)
        kl_b = self.kl_divergence(self.b_mean, b_std, prior_mean, prior_std)
        # kl of the bnn with a weight
        kl_loss = self.kl_weight*(kl_w + kl_b)
        return kl_loss


# BNN With flipout

In standard Bayesian Neural Networks (BNNs), the same random perturbation is often applied to all the data points within a mini-batch. This means that the sampled weights that are applied to each data point in the mini-batch are the same. This shared randomness leads to a correlation between the gradients computed for different data points in the mini-batch.

This correlation within the mini-batch does not, however, extend to different mini-batches. Each new mini-batch will have its own random perturbations applied to the weights, keeping them uncorrelated with other mini-batches. The challenge with this correlation is specifically confined to individual mini-batches.

Flipout is a technique designed to tackle this problem. By introducing unique random perturbations for each data point within the mini-batch, i.e. an efficient way to perturb the weights quasi-independently within a mini-batch. Flipout ensures that the gradients are decorrelated across the data points. This decorrelation helps to reduce the variance in the gradient estimates, thereby stabilizing the training process. 

### Step 1: Prepare the Data and Define Hyperparameters

First, you'll need to prepare the inputs, weights, and hyperparameters. Depending on your specific problem and network architecture, these will vary.

### Step 2: Define Random Sign Matrices

You'll need to define the random sign matrices $ R \) and \( S \), as described. These matrices have the same dimensions as your mini-batch size, and the entries are sampled uniformly from \(\{-1, 1\}\).

### Step 3: Define the Flipout Operation

Next, you'll need to define the operation for applying the Flipout estimator to a layer of your network. Using the equations provided:

$
Y = \phi \left( XW + \left( (X \circ S) \hat{\Delta W} \right) \circ R \right)
$

You can define this operation as a function:


### Step 4: Incorporate Flipout into Your Network

Now, you'll need to incorporate this Flipout operation into the architecture of your network. This might involve modifying existing layers or defining new custom layers that use the `flipout_layer` function.

### Step 5: Training and Backpropagation

Training the model with Flipout involves typical training loop steps, including forward and backward passes. Because the Flipout operation is differentiable, you can use standard optimization algorithms like Adam or SGD.

The full implementation details might vary based on your specific model architecture, problem domain, and the deep learning framework you are using. The provided explanation and code snippets should provide a starting point for integrating the Flipout estimator into your Bayesian Neural Network.

Note that if you are using TensorFlow Probability, there are built-in layers like `tfp.layers.DenseFlipout` that implement this functionality, so you might not need to manually implement it unless you have specific customization requirements.

Certainly! Below, I'll describe the mathematical equations corresponding to the code provided. These equations represent the Flipout estimator and detail how the weights and perturbations are handled.

1. **Sampling from the weight distribution**:
   The weight and bias terms are sampled from their respective distributions:
   \[
   \begin{align*}
   \mathbf{w} &\sim \mathcal{N}(\mathbf{w}_{\text{mean}}, \text{softplus}(\mathbf{w}_{\text{std}})) \\
   \mathbf{b} &\sim \mathcal{N}(\mathbf{b}_{\text{mean}}, \text{softplus}(\mathbf{b}_{\text{std}}))
   \end{align*}
   \]

2. **Random sign matrices (Rademacher distribution)**:
   \[
   \begin{align*}
   \mathbf{R} &\sim \text{Rademacher}(\text{shape}=[d_{\text{in}}, \text{units}]) \\
   \mathbf{S} &\sim \text{Rademacher}(\text{shape}=[d_{\text{in}}, \text{units}])
   \end{align*}
   \]
   where \(\text{Rademacher}\) denotes the Rademacher distribution, and \(d_{\text{in}}\) and \(\text{units}\) correspond to the dimensions of the input and the number of units in the dense layer, respectively.

3. **Base perturbation**:
   A base perturbation is drawn from a standard normal distribution:
   \[
   \hat{\Delta \mathbf{W}} \sim \mathcal{N}(0, 1)
   \]

4. **Application of Flipout**:
   The Flipout estimator is applied to incorporate the perturbation into the weights:
   \[
   \text{perturbation} = \left( \mathbf{x} \odot \mathbf{S} \right) \mathbf{\hat{\Delta W}} \odot \mathbf{R}
   \]
   where \(\odot\) denotes element-wise multiplication.

5. **Output calculation**:
   The output of the layer is computed by combining the sampled weights, perturbations, and biases, and applying the activation function (ReLU in this case):
   \[
   \text{output} = \text{ReLU}\left( \mathbf{x}\mathbf{w} + \text{perturbation} + \mathbf{b} \right)
   \]

These equations provide a mathematical representation of the Flipout technique as implemented in the given code, describing the stochastic behavior and variance reduction benefits of Flipout in Bayesian Neural Networks.